In [8]:
import pandas as pd
import hashlib
import os
import glob
from openpyxl import load_workbook

# === Path Setup ===
source_path = '/mnt/c/Bhavcopy/NSE_FundaMenal_Source/'
final_output_path = '/mnt/c/Bhavcopy/NSE_Funda/Temp_Quarter_result.xlsx'
sheet_name = 'Data Sheet'

# === Collect all .xlsx files in source_path
excel_files = glob.glob(os.path.join(source_path, '*.xlsx'))
write_header_once = not os.path.exists(final_output_path)

for file_path in excel_files:
    file_name = os.path.basename(file_path)
    print(f"\n📄 Processing: {file_name}")

    try:
        # Step 1: Read metadata from B1:B3
        meta_values = pd.read_excel(
            file_path,
            sheet_name=sheet_name,
            usecols='B',
            nrows=3,
            header=None
        ).squeeze()

        company_name = meta_values.iloc[0]
        latest_version = meta_values.iloc[1]
        current_version = meta_values.iloc[2]

        # Step 2: Read financials from rows 41–50, A–K
        df_raw = pd.read_excel(
            file_path,
            sheet_name=sheet_name,
            skiprows=40,
            nrows=10,
            usecols='A:K',
            header=None
        )

        # Step 3: Set column A as index
        df_raw.set_index(df_raw.columns[0], inplace=True)

        # Step 4: Transpose
        df_qtr = df_raw.T
        df_qtr.columns.name = None

        # Step 5: Extract actual report dates from row 41, B–K
        original_columns = pd.read_excel(
            file_path,
            sheet_name=sheet_name,
            skiprows=40,
            nrows=1,
            usecols='B:K',
            header=None
        ).iloc[0].tolist()

        report_dates = pd.to_datetime(original_columns, format='%b-%y', errors='coerce')

        # Step 6: Insert clean Report_Date
        df_qtr.insert(0, 'Report_Date', report_dates.date)

        # Step 7: Drop duplicate column if exists
        if 'Report Date' in df_qtr.columns:
            df_qtr.drop(columns=['Report Date'], inplace=True)

        # Step 8: Add metadata
        df_qtr.insert(1, 'Company_Name', company_name)
        df_qtr.insert(2, 'Latest_Version', latest_version)
        df_qtr.insert(3, 'Current_Version', current_version)

        # Step 9: Add MD5 hash
        def row_to_md5(row):
            row_string = '|'.join(str(val) for val in row.values)
            return hashlib.md5(row_string.encode('utf-8')).hexdigest()

        df_qtr['Row_Hash_MD5'] = df_qtr.apply(row_to_md5, axis=1)

        # Step 10: Append to Excel
        if write_header_once:
            df_qtr.to_excel(final_output_path, index=False)
            write_header_once = False
            print(f"✅ Created file and added {len(df_qtr)} rows.")
        else:
            with pd.ExcelWriter(final_output_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
                existing_wb = load_workbook(final_output_path)
                ws = existing_wb.active
                start_row = ws.max_row
                df_qtr.to_excel(writer, index=False, header=False, startrow=start_row)
                print(f"✅ Appended {len(df_qtr)} rows at row {start_row + 1}")

    except Exception as e:
        print(f"❌ Error in {file_name}: {e}")

# === Final Cleanup: Remove rows with empty Report_Date and Sales ===
try:
    df_final = pd.read_excel(final_output_path)

    # Ensure 'Report_Date' is datetime (handles first and second run)
    if 'Report_Date' in df_final.columns:
        df_final['Report_Date'] = pd.to_datetime(df_final['Report_Date'], errors='coerce')

    if 'Sales' in df_final.columns:
        initial_count = len(df_final)

        # Drop rows where both Report_Date and Sales are missing
        df_final = df_final[~(df_final['Report_Date'].isna() & df_final['Sales'].isna())]
        final_count = len(df_final)

        # Convert Report_Date to string before writing to Excel
        df_final['Report_Date'] = df_final['Report_Date'].dt.strftime('%Y-%m-%d')

        # Save cleaned file
        df_final.to_excel(final_output_path, index=False)
        print(f"\n🧹 Cleanup complete: Removed {initial_count - final_count} invalid rows.")

    else:
        print("⚠️ 'Sales' column missing — cleanup skipped.")

except Exception as e:
    print(f"❌ Cleanup error: {e}")






📄 Processing: All E Tech.xlsx
✅ Created file and added 10 rows.

📄 Processing: Allied Blenders.xlsx
✅ Appended 10 rows at row 12

📄 Processing: Anand Rathi Wea.xlsx
✅ Appended 10 rows at row 22

📄 Processing: Anant Raj.xlsx
✅ Appended 10 rows at row 32

📄 Processing: Apar Inds.xlsx
✅ Appended 10 rows at row 42

📄 Processing: Blue Jet Health.xlsx
✅ Appended 10 rows at row 52

📄 Processing: Caplin Point Lab.xlsx
✅ Appended 10 rows at row 62

📄 Processing: CEINSYS Tech.xlsx
✅ Appended 10 rows at row 72

📄 Processing: Data Pattern.xlsx
✅ Appended 10 rows at row 82

📄 Processing: E2E Networks.xlsx
✅ Appended 10 rows at row 92

📄 Processing: Fidel Softech.xlsx
✅ Appended 10 rows at row 102

📄 Processing: FSN E-Commerce.xlsx
✅ Appended 10 rows at row 112

📄 Processing: Genus Power.xlsx
✅ Appended 10 rows at row 122

📄 Processing: Goldiam Intl.xlsx
✅ Appended 10 rows at row 132

📄 Processing: Indrapr.Medical.xlsx
✅ Appended 10 rows at row 142

📄 Processing: KPI Green Energy.xlsx
✅ Appended 10